In [34]:
import os 
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [35]:
PATH_CSV_LABELS = 'tabellaPseudoLabelsDatasetCompleto.csv'
PATH_DATASET_LAVORATO = '../Nuovi Dati/TemplateGeneratiClean'

PATH_MODELLO_SPECKLE = '../Modelli-v2/modello_finale_riaddestrato_speckle'
PATH_MODELLO_GAUSSIAN_BLUR = '../Modelli-v2/modello_finale_riaddestrato_gaussian_blur'

N_IMG = 1
FORMATO_IMG = 'jpg'
CHANNELS = 1
HEIGHT = 512
WIDTH = 512
TARGET_SIZE = (WIDTH, HEIGHT)

In [36]:
df_labels = pd.read_csv(PATH_CSV_LABELS)
df_labels.head(5)

,Acquisizione,Label,Label_norm
0,AgatielloRoberto_000,0.648876,50.428711
1,AgatielloRoberto_001,0.759503,61.300294
2,AgatielloRoberto_002,0.747500,60.120716
3,AgatielloRoberto_003,0.818899,67.137182
4,AgatielloRoberto_004,0.825895,67.824714


In [68]:
X = []
y = []

for id_acquisizione in df_labels.Acquisizione:
    path_img = os.path.join(PATH_DATASET_LAVORATO, id_acquisizione, 'Immagini', 'immagine1.jpg')
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [60]:
test_set = df_labels.Acquisizione.apply(lambda x:  x.startswith('MarcheseLuigi'))
test_set = df_labels[test_set]

In [63]:
X = []
y = []

for id_acquisizione in test_set.Acquisizione.unique():
    path_img = os.path.join(PATH_DATASET_LAVORATO, id_acquisizione, 'Immagini', 'immagine1.jpg')
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [69]:
model_speckle = tf.keras.models.load_model(PATH_MODELLO_SPECKLE)
model_gaussian_blur = tf.keras.models.load_model(PATH_MODELLO_GAUSSIAN_BLUR)

In [70]:
previsioni_nuovo_modello = model_speckle.predict(X)
# previsioni_nuovo_modello = model_gaussian_blur.predict(X)

 1/23 [>.............................] - ETA: 3s

2023-07-25 18:02:11.066144: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 3s 137ms/step


In [75]:
lista_acquisizioni = list(df_labels[previsioni_nuovo_modello < 34].Acquisizione.unique())

In [76]:
PATH_FILE_SCORE = '../scoreCsvDatasetFinale.csv'
scores = pd.read_csv(PATH_FILE_SCORE)
scores = scores[~scores.Utente1.isin(lista_acquisizioni)]
scores = scores[~scores.Utente2.isin(lista_acquisizioni)]
scores

,Utente1,Utente2,ScoreML
0,AgatielloRoberto_000,AgatielloRoberto_001,0.526569
1,AgatielloRoberto_000,AgatielloRoberto_002,0.491637
2,AgatielloRoberto_000,AgatielloRoberto_003,0.525314
3,AgatielloRoberto_000,AgatielloRoberto_004,0.516008
4,AgatielloRoberto_000,AgatielloRoberto_005,0.567200
...,...,...,...
258116,VenafraSara_012,VenafraSara_014,0.593949
258117,VenafraSara_012,VenturaAlessio_000,0.121388
258118,VenafraSara_013,VenafraSara_014,0.599829
258119,VenafraSara_013,VenturaAlessio_000,0.121099


In [52]:
scores.to_csv('../scoreFiltratiFinaliCsv.csv' ,index=False)